# Advertising events in the MTA Stations 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [184]:
import config
#SQL
import mysql.connector
from mysql.connector import errorcode
from sqlalchemy import create_engine

In [206]:
cnx = mysql.connector.connect(
 host = config.host,
 user = config.user,
 password = config.passwd)
print(cnx)
cursor = cnx.cursor()
#insert Database Name
db_name = 'mtaevents'

InterfaceError: 2003: Can't connect to MySQL server on '%-.100s:%u' (%s) (Warning: %u format: a number is required, not str)

In [ ]:
#creates db
def create_database(cursor, database):
     try:
        cursor.execute(
         "CREATE DATABASE {} DEFAULT CHARACTER SET
        'utf8'".format(database))
         except mysql.connector.Error as err:
         print("Failed creating database: {}".format(err))
         exit(1)
    try:
         cursor.execute("USE {}".format(db_name))
    except mysql.connector.Error as err:
         print("Database {} does not exists.".format(db_name))
         if err.errno == errorcode.ER_BAD_DB_ERROR:
         create_database(cursor, db_name)
         print("Database {} created successfully.".format(db_name))
         cnx.database = db_name
         else:
         print(err)
         exit(1)

### Creating a function to concat the MTA turnstile datasets into one dataframe

In [2]:
def concant(weeks):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    
    dfs = []
    for i in weeks:
        url_file = url.format(i)
        print(url_file)
        dfs.append(pd.read_csv(url_file))
    return pd.concat(dfs)

### Generate the wanted started date for getting the MTA turnstile datasets

In [3]:
from datetime import datetime, timedelta

def generate_dates (sdate):
    dates = pd.date_range(sdate,pd.to_datetime("today")-timedelta(days=1),freq='w-sat')
    weeks = [i.strftime('%y%m%d') for i in list(dates)]

    return concant(weeks)

In [ ]:
df = generate_dates('2021-06-27')
df.head()

In [ ]:
df.dtypes

### Convert Date attribute to datetime object

In [ ]:
df['DATE'] = df['DATE'].apply(pd.to_datetime)
df['DATE'].head()

### Read NYC events dataset


In [ ]:
df_event = pd.read_csv("NYC_Permitted_Event_Information.csv")
df_event['Start Date/Time']

##### Find the percentage of missing values in each column in the event dataset


In [ ]:
for col in df_event.columns:
    pct_missing = np.mean(df_event[col].isna())
    print('{} - {}%'.format(col, round(pct_missing*100)))

##### Drop columns with missing values


In [ ]:
df_event.columns
df_event = df_event.drop(labels=['Event Street Side','Street Closure Type'],axis=1)
df_event.columns

##### Checking the type of each column


In [ ]:
df_event.dtypes

##### Convert Start and End date to datetime objects

In [13]:
df_event[['Start Date/Time','End Date/Time']] = df_event[['Start Date/Time','End Date/Time']].apply(pd.to_datetime)

In [ ]:
df_event.dtypes

#### Extract only the date from Start Date/Time attribute

In [ ]:
df_event['Start Date'] = df_event['Start Date/Time'].dt.date
df_event['Start Date']

### Create a mask range for events that occurred over a certain time period.

In [ ]:

mask = (df_event["Start Date"] > pd.Timestamp('2021-06-27')) & (df_event["Start Date"] <= pd.Timestamp('2021-09-27'))
df_event_range = df_event.loc[mask]
df_event_range = df_event_range.sort_values(by='Start Date')
df_event_range



#### Convert Start Date attribute to datetime object

In [18]:
df_event_range['Start Date'] = df_event_range['Start Date'].apply(pd.to_datetime)

#### Check the most type of events that happened in each month

In [ ]:
date_range = pd.date_range(start='2021-06-27', end='2021-09-27',freq='M')
print(date_range)
most_event_type = df_event_range.groupby([df_event_range['Start Date'].dt.month, 'Event Type'])['Event Type'].count()
most_event_type = pd.DataFrame(most_event_type)
most_event_type = most_event_type.rename({'Event Type' : 'Count' , 'Start Date' : 'Month'},axis=1)
print(most_event_type.columns)
most_event_type

#### Changing the name of the rows to Month name instead of month number and resetting the index of the dataframe

In [ ]:
indexed_most_event_type = most_event_type.reset_index()
indexed_most_event_type = indexed_most_event_type.rename({'Start Date' : 'Month'},axis=1)
indexed_most_event_type['Month'] = indexed_most_event_type['Month'].replace({7:'July',8:'August',9:'September '})
indexed_most_event_type


In [ ]:
for col in df_event.columns:
    pct_missing = np.mean(df_event[col].isna())
    print('{} - {}%'.format(col, round(pct_missing*100)))

#### Check how many unique event locations

In [23]:
event_locations = df_event['Event Location'].unique()
print(len(event_locations))

3523


#### Getting unique event types 

In [ ]:
df_event["Event Type"] = df_event["Event Type"].str.strip(' ')
event_type = df_event["Event Type"].unique()
print(event_type)


#### Most common date for events

In [ ]:
events_date = df_event_range['Start Date/Time'].dt.date.value_counts()
events_date

In [ ]:
# events_date_plot = events_date.plot(kind='pie', figsize=(10,10),autopct='%1.0f%%')
# events_date.to_csv('events_date.csv')

#### How many events have occured in each date

In [ ]:
grouped_by_date = df_event_range.groupby(df_event_range['Start Date'])["Event Type"].count()
grouped_by_date = pd.DataFrame(grouped_by_date)
grouped_by_date = grouped_by_date.sort_values('Start Date')
grouped_by_date

#### Checking event boroughs

In [ ]:
boroughs = df_event_range['Event Borough'].unique()
boroughs

#### How many events have occured in total

In [ ]:
events_type_count = df_event_range["Event Type"].value_counts()
events_type_count

#### Type of events and how many each occured in Brooklyn

In [ ]:

brooklyn_grouped_by = df_event_range[df_event_range['Event Borough'] == 'Brooklyn'].groupby(['Event Borough'])['Event Type'].value_counts()
df_brooklyn_grouped_by = pd.DataFrame(brooklyn_grouped_by)
df_brooklyn_grouped_by = df_brooklyn_grouped_by.rename({'Event Type': 'Count'},axis=1)
df_brooklyn_grouped_by



#### A simple visualization of event types that were placed in Brooklyn

In [ ]:
df_brooklyn_grouped_by_plot = df_brooklyn_grouped_by.plot.barh(title="Brooklyn - Event Types");
plt.savefig('df_brooklyn_grouped_by_plot.png', dpi=300, bbox_inches='tight')


#### Type of events and how many each occured in Manhattan

In [ ]:
manhattan_grouped_by = df_event_range[df_event_range['Event Borough'] == 'Manhattan'].groupby(['Event Borough'])['Event Type'].value_counts()
df_manhattan_grouped_by = pd.DataFrame(manhattan_grouped_by)
df_manhattan_grouped_by = df_manhattan_grouped_by.rename({'Event Type': 'Count'},axis=1)
df_manhattan_grouped_by.sort_values(by='Count', ascending=False)

In [ ]:
df_manhattan_grouped_by.plot.barh(title=" Manhattan - Event Types");

##### Type of events and how many each occured in the other boroughs

In [ ]:
queens_grouped_by = df_event_range[df_event_range['Event Borough'] == 'Queens'].groupby(['Event Borough'])['Event Type'].value_counts()
df_queens_grouped_by = pd.DataFrame(queens_grouped_by)
df_queens_grouped_by = df_queens_grouped_by.rename({'Event Type': 'Count'},axis=1)
df_queens_grouped_by.sort_values(by='Count', ascending=False)

In [ ]:
bronx_grouped_by = df_event_range[df_event_range['Event Borough'] == 'Bronx'].groupby(['Event Borough'])['Event Type'].value_counts()
df_bronx_grouped_by = pd.DataFrame(bronx_grouped_by)
df_bronx_grouped_by = df_bronx_grouped_by.rename({'Event Type': 'Count'},axis=1)
df_bronx_grouped_by.sort_values(by='Count', ascending=False)

In [ ]:
staten_island_grouped_by = df_event_range[df_event_range['Event Borough'] == 'Staten Island'].groupby(['Event Borough'])['Event Type'].value_counts()
df_staten_island_grouped_by = pd.DataFrame(staten_island_grouped_by)
df_staten_island_grouped_by = df_staten_island_grouped_by.rename({'Event Type': 'Count'},axis=1)
df_staten_island_grouped_by.sort_values(by='Count', ascending=False)

In [40]:
turnstiles_df = df

### Reading the MTA Station dataset

In [ ]:
stations = pd.read_csv("http://web.mta.info/developers/data/nyct/subway/Stations.csv")
stations.head()

#### Unifying station name from the MTA Station dataset and MTA turnstile dataset

In [42]:
stations["STATION"] = stations["Stop Name"].str.strip().str.strip(".").str.upper()
stations.drop_duplicates(subset ="STATION",
                     keep = "first", inplace = True)
stations = stations.sort_values("STATION")
turnstiles_df = turnstiles_df.sort_values("STATION")


In [43]:
turnstiles_df.drop(turnstiles_df.index[turnstiles_df['STATION'] == '14TH STREET'], inplace = True)
turnstiles_df["STATION"] = turnstiles_df["STATION"].str.strip().str.strip(".").str.upper()
stations["STATION"] = turnstiles_df["STATION"].unique().copy()

In [44]:
stations_borough = stations[["STATION", "Borough"]]
stations_borough = stations_borough.replace({"Borough" : {"M" : "Manhattan", "Q" : "Queens",
                                                          "Bx" : "Bronx", "Bk" : "Brooklyn",
                                                          "SI" : "Staten Island"}}, regex = True)


#### Merging both MTA station dataset and MTA turnstile dataset on STATION

In [ ]:
stations_borough_totals = pd.merge(turnstiles_df, stations_borough, on = "STATION")
stations_borough_totals

In [ ]:
# stations_borough_totals_mini = stations_borough_totals[['STATION','Borough','ENTRIES','DATE']].copy()
# stations_borough_totals_mini
# df_event_range_mini = df_event_range[['Event Name','Event Type','Event Borough','Start Date']].copy()
# df_event_range_mini

#### Getting the total entries in each borough for the last 3 months

In [ ]:
total_entries_borough = stations_borough_totals.groupby(['Borough'])['ENTRIES'].sum()
total_entries_borough


In [ ]:
plt.title("Total entries for each borough for the last 3 months",fontsize=20)
total_entries_borough_image = total_entries_borough.plot(kind='bar',figsize=(20,10),fontsize=16)

plt.savefig('total_entries_borough_image.png', dpi=300, bbox_inches='tight')



#### Getting top stations by number of entries

In [191]:
top_stations = df.groupby(['STATION'], as_index = False )['ENTRIES'].max()
top_stations = pd.DataFrame(top_stations)
top_stations = top_stations.sort_values('ENTRIES', ascending=False)

top_stations.to_csv('top_stations.csv')

In [ ]:
top_stations